In [33]:
# Ahmed Tarek Abdellatif 1190157 
# Yep ! I am alone , and built it from scratch !! 

In [34]:
# Classes of Components
class Voltage_Source:
    def __init__(self, initial_value, terminal1, terminal2, id):
        self.ini_val = float(initial_value)
        self.pnode = int(terminal1)
        self.nnode = int(terminal2)
        self.name = "V"+str(id)

    def __str__(self):
        return self.name


class Current_Source():
    def __init__(self, initial_value, terminal1, terminal2, id):
        self.ini_val = float(initial_value)
        self.pnode = int(terminal1)
        self.nnode = int(terminal2)
        self.name = "I"+str(id)
    def __str__(self):
        return self.name

class Conductance():
    def __init__(self,value,terminal1, terminal2, id):
        self.ini_val = 1/float(value)
        self.pnode = int(terminal1)
        self.nnode = int(terminal2)
        self.name = "R"+str(id)
    def __str__(self):
        return self.name

class Capacitance():
    def __init__(self,value,ini_voltage,terminal1, terminal2, id,step_size):
        self.h = step_size
        self.ini_val = float(value)/self.h
        self.V_ini = float(ini_voltage)
        self.pnode = int(terminal1)
        self.nnode = int(terminal2)
        self.name = "C"+str(id)
    def __str__(self):
        return self.name

class Inductance():
    def __init__(self,value,ini_curr,terminal1, terminal2, id,step_size):
        self.h = step_size
        self.ini_val = float(value) / self.h
        self.I_ini = float(ini_curr)
        self.pnode = int(terminal1)
        self.nnode = int(terminal2)
        self.name = "L"+str(id)
    def __str__(self):
        return self.name


In [35]:
import os
import numpy as np
from decimal import Decimal

def print_list(x):
    print("Items in list are")
    for item in x:
        print(item)
    return


def get_terminal_number(terminal):
    # A function to return the node index to be assigned in the MNA stamp
    terminal = terminal.split("V", 1)[1]
    return terminal

def Update_Matrix_Z(X_new,Capacitances,Isrcs,Vsrcs,Inductances):
    #re-initialize the Z with zeros 
    Z_old = np.zeros((X_new.size,1))

    count_nodes = int(X_new.size) - len(Vsrcs) - len(Inductances)
    #iterate over current sources 
    for isrc in Isrcs:
        if isrc.nnode == 0:
            Z_old[isrc.pnode - 1] += isrc.ini_val
        else:
            Z_old[isrc.pnode - 1] += isrc.ini_val
            Z_old[isrc.nnode - 1] -= isrc.ini_val
    #Iterate over capacitances        
    for capacitance in Capacitances:
        capacitance.V_ini = X_new[capacitance.pnode -1][0]
        if capacitance.nnode == 0:
            Z_old[capacitance.pnode -1] += capacitance.ini_val * capacitance.V_ini
        else:
            capacitance.V_ini = X_new[capacitance.pnode -1][0] - X_new[capacitance.nnode -1][0] 
            Z_old[capacitance.pnode -1] += capacitance.ini_val * capacitance.V_ini
            Z_old[capacitance.nnode -1] -= capacitance.ini_val * capacitance.V_ini
    #Finally iterate over Vsrcs and indices
    i=0
    for vsrc in Vsrcs:
        if vsrc.nnode == 0:
            Z_old[count_nodes + i] += vsrc.ini_val
            i+=1
        else:
            Z_old[count_nodes + i] += vsrc.ini_val
            i=i+1

    for inductance in Inductances:
        inductance.I_ini = X_new[count_nodes + i][0]
        Z_old[count_nodes + i] -= inductance.ini_val * inductance.I_ini
        i+=1
    return Z_old



def Initialize_Circuit(elements, h , iterations):
    # I need to know how many voltage sources do I have , How many Isrcs , How many nodes , How many RLC components
    Isrcs = []
    Vsrcs = []
    Conductances = []
    Capacitances = []
    Inductances = []
    nodes = []
    v = []
    jj = []
    component_id = -1
    h = float(h)
    iterations = int(iterations)
    # C/h and L/h are calculated within
    for item in elements:
        argV = item.split(" ", 5)
        component_id += 1
        if argV[0] == "Isrc":
            Isrcs.append(Current_Source(argV[3], get_terminal_number(
                argV[1]), get_terminal_number(argV[2]), component_id))
        elif argV[0] == "Vsrc":
            Vsrcs.append(Voltage_Source(argV[3], get_terminal_number(
                argV[1]), get_terminal_number(argV[2]), component_id))
        elif argV[0] == "R":
            Conductances.append(Conductance(argV[3], get_terminal_number(
                argV[1]), get_terminal_number(argV[2]), component_id))
        elif argV[0] == "C":
            Capacitances.append(Capacitance(argV[3],argV[4],get_terminal_number(
                argV[1]), get_terminal_number(argV[2]), component_id,h))
        else:
            Inductances.append(Inductance(argV[3], argV[4],get_terminal_number(
                argV[1]), get_terminal_number(argV[2]), component_id,h))
        nodes.append(argV[1])
        nodes.append(argV[2])

    # It is to filter the nodes list and remove duplicates
    count_nodes = len(list(dict.fromkeys(nodes)))
    # To ignore ground
    count_nodes -= 1


    # Initialzing Unknown variables  Names
    for i in range(count_nodes):
        v.append("V" + str(i+1))

    for i in range(len(Vsrcs)):
        jj.append("I_Vsrc"+str(i))

    for i in range(len(Inductances)):
        jj.append("I_L" + str(i))

    # Intializing the Possible Matrices 

    G = np.zeros((count_nodes, count_nodes))
    B = np.zeros((count_nodes,len(Vsrcs)+len(Inductances)))
    Z = np.zeros((count_nodes + len(Vsrcs) + len(Inductances) ,1))

    # starting with Conductances
    for conductance in Conductances:
        if conductance.pnode != 0 and conductance.nnode == 0:
            G[conductance.pnode - 1][conductance.pnode - 1] += conductance.ini_val
        else:
            # the case where you have different pnode and nnode
            G[conductance.pnode - 1][conductance.pnode - 1] += conductance.ini_val
            G[conductance.pnode - 1][conductance.nnode - 1] -= conductance.ini_val
            G[conductance.nnode - 1][conductance.pnode - 1] -= conductance.ini_val
            G[conductance.nnode - 1][conductance.nnode - 1] += conductance.ini_val


    # Now with Capacitances , Notice that C fills the Z first before Vsrcs because of Z is composed of i and e
  
    for capacitance in Capacitances:
        if capacitance.pnode != 0 and capacitance.nnode == 0:
            G[capacitance.pnode - 1][capacitance.pnode - 1] += capacitance.ini_val
            Z[capacitance.pnode - 1] += (capacitance.ini_val) * (capacitance.V_ini)
        else:
            # the case where you have different pnode and nnode
            G[capacitance.pnode -1][capacitance.pnode -1] += capacitance.ini_val
            G[capacitance.pnode -1][capacitance.nnode -1] -= capacitance.ini_val
            G[capacitance.nnode -1][capacitance.pnode -1] -= capacitance.ini_val
            G[capacitance.nnode -1][capacitance.nnode -1] += capacitance.ini_val
            Z[capacitance.pnode - 1] += (capacitance.ini_val) * (capacitance.V_ini)
            Z[capacitance.nnode - 1] -= (capacitance.ini_val) * (capacitance.V_ini)

    # Voltage Sources MNA stamp 
    i=0 #for Z initialization 
    j=0 #for B initialzation
    for vsrc in Vsrcs:
        if vsrc.pnode == 0 and vsrc.nnode != 0:
            B[vsrc.nnode -1 ][j] = -1
            Z[count_nodes + i] += vsrc.ini_val
            i+=1
            j+=1
        elif vsrc.nnode == 0 and vsrc.pnode != 0:
            B[vsrc.pnode -1][i] = 1
            Z[count_nodes + i] += vsrc.ini_val
            i+=1
            j+=1
        else:
            B[vsrc.pnode -1][i] = 1
            B[vsrc.nnode -1][i] = -1
            Z[count_nodes + i] += vsrc.ini_val
            i+=1
            j+=1


    D = np.zeros((len(Vsrcs)+len(Inductances),len(Vsrcs)+len(Inductances)))
    # Now with Inductances , Notice that L fills B (C by transpose) , D and Z (after the voltage sources)

    for inductance in Inductances:
        if inductance.nnode == 0: 
            B[inductance.pnode -1][j] = 1
            Z[count_nodes + i] -=  inductance.ini_val * inductance.I_ini
            D[j][j] -= inductance.ini_val
            j+=1
            i+=1
        else:
            B[inductance.pnode -1][j] = 1
            B[inductance.nnode -1][j] = -1
            Z[count_nodes + i] -=  inductance.ini_val * inductance.I_ini
            D[j][j] -= inductance.ini_val
            j+=1
            i+=1


    # Current Source MNA stamp 

    for isrc in Isrcs:
        if isrc.pnode == 0 and isrc.nnode != 0:
            Z[isrc.nnode -1] -= isrc.ini_val
        elif isrc.nnode == 0 and isrc.pnode != 0:
            Z[isrc.pnode -1] += isrc.ini_val
        else:
            Z[isrc.pnode -1] += isrc.ini_val
            Z[isrc.nnode -1] -= isrc.ini_val 

    # Creating the A matrix 
    C = np.transpose(B)
    A = np.block([[G,B],[C,D]])
    
    
    if np.linalg.det(A) == 0:
        print(A)
        print("Singular Matrix")
        return False

 
    #Computing resultant X matrix 
    out_file = input("Enter file name to write to : ")
    f = open(out_file,"w")
    if not Inductances and not Capacitances:
        X = np.matmul(np.linalg.inv(A) , Z)
        for i in range(len(v)+len(j)):
            if i < len(v):
                f.write(v[i])
                f.write(" \n")
                f.write(str(h)+"    ".rstrip('\n'))
                f.write(str(X[i][0]))
                f.write(" \n")
                f.write(" \n")
            else:
                if i==len(v): f.write('\n')
                f.write(jj[i-len(v)])
                f.write(" \n")
                f.write(str(h)+"    ".rstrip('\n'))
                f.write(str(X[i][0]))
                f.write(" \n")
        f.close()
    else:
        #Create a matrices to save X values 
        table = []
        #Iterate 
        for i in range(iterations):
            X = np.matmul(np.linalg.inv(A) , Z)
            table.append(X)
            print("X[" +str(i+1) + "]" , X)
            Z = Update_Matrix_Z(X,Capacitances,Isrcs,Vsrcs,Inductances)               
        #write to file
        pcount = 1 #print_count(used for formatting)
        step = Decimal(h).quantize(Decimal('.1'))
        for col in range(X.size):
            h = Decimal(step).quantize(Decimal('.1'))
            for row in range(iterations):
                if col < len(v):
                    if row==0: 
                        f.write(v[col])
                        f.write('\n')
                    f.write(str(h)+"    ".rstrip('\n'))
                    f.write(str(table[row][col]).strip('[]'))
                    f.write("\n")
                    if row==(iterations-1): f.write("\n")
                else:
                    #for Isrc
                    if col == len(v) and pcount==1:
                        f.write("\n")
                        f.write(jj[col-len(v)])
                        pcount+=1
                    #for I_L
                    if col == len(v) + len(Vsrcs) and pcount ==2 and len(Vsrcs) != 0:
                        f.write("\n")
                        f.write(jj[col-len(v)])
                        pcount+=1
                    f.write("\n")
                    f.write(str(h)+"    ".rstrip('\n'))
                    f.write(str(table[row][col]).strip('[]'))
                    if row==(iterations-1): f.write("\n")
                h += step
        f.close()
    return True


In [36]:
# MAIN
components = []
filepath = input("Enter file name to parse: ")
if os.path.exists(filepath):
    f = open(filepath, "r")
    h = f.readline()
    iterations = f.readline()
    eof = f.readline()
    while True:
        if ("-1" == eof):
            break
        components.append(eof)
        eof = f.readline()
    Initialize_Circuit(components,h,iterations)

else:
    # Print message if the file does not exist
    print("File does not exist.")


X[1] [[0.08196721]
 [0.98360656]
 [0.09836066]]
X[2] [[0.14780973]
 [0.95404461]
 [0.19376512]]
X[3] [[0.19935589]
 [0.91417343]
 [0.28518246]]
X[4] [[0.23833872]
 [0.86650569]
 [0.37183303]]
X[5] [[0.26638466]
 [0.81322876]
 [0.4531559 ]]
X[6] [[0.28500618]
 [0.75622752]
 [0.52877866]]
X[7] [[0.29559748]
 [0.69710802]
 [0.59848946]]
X[8] [[0.29943302]
 [0.63722142]
 [0.6622116 ]]
X[9] [[0.29766817]
 [0.57768779]
 [0.71998038]]
X[10] [[0.29134176]
 [0.51941944]
 [0.77192232]]
X[11] [[0.28138008]
 [0.46314342]
 [0.81823667]]
X[12] [[0.26860199]
 [0.40942302]
 [0.85917897]]
X[13] [[0.25372483]
 [0.35867806]
 [0.89504677]]
X[14] [[0.23737101]
 [0.31120385]
 [0.92616716]]
X[15] [[0.22007492]
 [0.26718887]
 [0.95288605]]
X[16] [[0.20229   ]
 [0.22673087]
 [0.97555913]]
X[17] [[0.18439597]
 [0.18985167]
 [0.9945443 ]]
X[18] [[0.16670585]
 [0.1565105 ]
 [1.01019535]]
X[19] [[0.14947287]
 [0.12661593]
 [1.02285694]]
X[20] [[0.1328971 ]
 [0.10003651]
 [1.03286059]]
